In [6]:
import tkinter as tk

In [7]:
class BinaryNode:
    indent = '  '
    node_radius = 10    # Radius of a node's circle
    x_spacing = 20      # Horizontal distance between neighbouring subtrees
    y_spacing = 20      # Vertical distance between parent and child subtrees
    
    def __init__(self, value=''):
        self.value = value
        self.left_child = None
        self.right_child = None
        
        # Initialise drawing parameters
        self.centre = (0, 0)
        self.subtree_bounds = (
            self.centre[0] - self.node_radius,
            self.centre[1] - self.node_radius,
            self.centre[0] + self.node_radius,
            self.centre[1] + self.node_radius
        )

    def __str__(self, level=0):
        return (
            f"{self._indent_node(self.value, level=level)}:\n"
            f"{self.left_child.__str__(level=level+1) if self.left_child else self._indent_missing_child_node('None', level=level+1)}"
            f"{self.right_child.__str__(level=level+1) if self.right_child else self._indent_missing_child_node('None', level=level+1)}"
        )

    def _indent_missing_child_node(self, value='', level=0):
        """
        Prepares a string indented to reflect the level of the empty child of the current node. 
        :param value: the text, if available, to include after the indentation 
        :param level: an integer to reflect the level (or depth) in the tree and the indentation required for the node
        :return: string containing enough indentation chars (usually spaces) followed by a text value if available
        """
        return self._indent_node(value + '\n', level=level) if self.left_child or self.right_child else ''

    @classmethod
    def _indent_node(cls, value='', level=0):
        """
        Prepares a string indented to reflect the level of the current node in the tree. 
        :param value: the text, if available, to include after the indentation
        :param level: an integer to reflect the level (or depth) in the tree and the indentation required for the node
        :return: string containing enough indentation chars (usually spaces) followed by a text value if available
        """
        return "".join([cls.indent for _ in range(0, level)]) + value

    def add_left(self, child):
        self.left_child = child

    def add_right(self, child):
        self.right_child = child
        
    def find_node(self, value):
        """
        Return the (first) node that has the corresponding value, starting from the current node
        and then traversing child nodes.
        :param value: the value to compare against each node
        :return: the node that matches the given value
        """
        if self.value == value:
            return self
        
        for child in [self.left_child, self.right_child]:
            node = child.find_node(value) if child else None
            if node:
                return node
        return None

    def traverse_preorder(self):
        nodes = [self]
        if self.left_child:
            nodes.extend(self.left_child.traverse_preorder())
        if self.right_child:
            nodes.extend(self.right_child.traverse_preorder())
        return nodes
    
    def traverse_inorder(self):
        nodes = self.left_child.traverse_inorder() if self.left_child else []
        nodes.extend([self])
        if self.right_child:
            nodes.extend(self.right_child.traverse_inorder())
        return nodes
    
    def traverse_postorder(self):
        nodes = self.left_child.traverse_postorder() if self.left_child else []
        if self.right_child:
            nodes.extend(self.right_child.traverse_postorder())
        nodes.extend([self])
        return nodes
    
    def traverse_breadth_first(self):
        result = []
        queue = [self]
        
        while queue:
            node = queue.pop(0)
            result.append(node)
            
            for c in [node.left_child, node.right_child]:
                queue.append(c) if c else None
        
        return result

    def arrange_subtree(self, xmin, ymin):
        """Position the node's subtree"""
        # Calculate cy, the Y coordinate for this node.
        cy = ymin + self.node_radius
        
        # If the node has no children, just place it here and return.
        if not self.left_child and not self.right_child:
            self.centre = (xmin + self.node_radius, cy)
            self.subtree_bounds = (
                xmin,
                ymin,
                self.centre[0] + self.node_radius,
                self.centre[1] + self.node_radius
            )
            return
        
        # Set child_xmin and child_ymin to the
        # start position for child subtrees.
        child_xmin = xmin
        child_ymin = cy + self.node_radius + self.y_spacing
        
        # Position the child subtrees
        if self.left_child:
            self.left_child.arrange_subtree(child_xmin, child_ymin)
            
            # Update positions for right subtrees if any
            child_xmin = self.left_child.subtree_bounds[2] + self.x_spacing
        
        if self.right_child:
            self.right_child.arrange_subtree(child_xmin, child_ymin)
        
        # Arrange this node depending on the number of children
        if self.left_child and self.right_child:
            # Two children. Centre this node over the child nodes.
            self.centre = ((self.left_child.centre[0] + self.right_child.centre[0]) / 2, cy) 
            # Use the child subtree bounds to set our subtree bounds.
            self.subtree_bounds = (
                xmin,
                ymin,
                self.right_child.subtree_bounds[2],
                max(self.left_child.subtree_bounds[3], self.right_child.subtree_bounds[3])
            )
        elif self.left_child:
            # We have only a left child
            self.centre = (self.left_child.centre[0], cy)
            self.subtree_bounds = (
                xmin,
                ymin,
                self.left_child.subtree_bounds[2],
                self.left_child.subtree_bounds[3]
            )
        else:
            # We have only a right child
            self.centre = (self.right_child.centre[0], cy)
            self.subtree_bounds = (
                xmin,
                ymin,
                self.right_child.subtree_bounds[2],
                self.right_child.subtree_bounds[3]
            )
    
    def draw_subtree_links(self, canvas):
        """Draw the subtree's links"""
        if self.left_child:
            canvas.create_line(self.centre, self.left_child.centre)
            self.left_child.draw_subtree_links(canvas)
        if self.right_child:
            canvas.create_line(self.centre, self.right_child.centre)
            self.right_child.draw_subtree_links(canvas)
            
        # Outline the subtree for debugging
        #canvas.create_rectangle(self.subtree_bounds, fill='', outline='red')
        
    def draw_subtree_nodes(self, canvas):
        """Draw the subtree's nodes"""
        # Draw the node
        canvas.create_oval(
            self.centre[0] - self.node_radius, 
            self.centre[1] - self.node_radius, 
            self.centre[0] + self.node_radius, 
            self.centre[1] + self.node_radius, 
            fill='white',
            outline='green'
        )
        canvas.create_text(self.centre, text=self.value, fill='red')
        
        # Draw the descendant's nodes
        if self.left_child:
            self.left_child.draw_subtree_nodes(canvas)
        if self.right_child:
            self.right_child.draw_subtree_nodes(canvas)
        
    def arrange_and_draw_subtree(self, canvas, xmin, ymin):
        # Position the tree
        self.arrange_subtree(xmin, ymin)
        
        # Draw the links
        self.draw_subtree_links(canvas)
        
        # Draw the nodes
        self.draw_subtree_nodes(canvas)


In [8]:
# Build a test tree.
#         A
#        / \
#       /   \
#      /     \
#     B       C
#    / \     / \
#   D   E   F   G
#      / \     /
#     H   I   J
#            / \
#           K   L
a = BinaryNode('A')
b = BinaryNode('B')
c = BinaryNode('C')
d = BinaryNode('D')
e = BinaryNode('E')
f = BinaryNode('F')
g = BinaryNode('G')
h = BinaryNode('H')
i = BinaryNode('I')
j = BinaryNode('J')
k = BinaryNode('K')
l = BinaryNode('L')

a.add_left(b)
a.add_right(c)
b.add_left(d)
b.add_right(e)
c.add_left(f)
c.add_right(g)
e.add_left(h)
e.add_right(i)
g.add_left(j)
j.add_left(k)
j.add_right(l)


In [9]:
def kill_callback():
    """A callback to destroy the tkinter window."""
    window.destroy()


In [10]:
window = tk.Tk()
window.title('binary_node5')
window.protocol('WM_DELETE_WINDOW', kill_callback)
window.geometry('260x220')

canvas = tk.Canvas(window, bg='white', borderwidth=2, relief=tk.SUNKEN)
canvas.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Draw the tree
a.arrange_and_draw_subtree(canvas, 10, 10)

window.focus_force()
window.mainloop()